# Setting up the initial simulation grid


### Emulator parameters $M$

The relation between the likelihood parameters ($\Theta$,$\Phi$) and the emulator parameters $M$ is discussed in the notebook full_likelihood. The emulator will have training set containing a large number of simulated spectra, identified by a list of parameters $M$:
$$ M = \{ M_P, f, \lambda_F, \bar F, \sigma_T, \gamma \} $$
where $M_P$ is a set of (3) parameters describing the linear power in the snapshot, in units of Mpc, $f$ allows us to translate that to the velocity power spectrum, and $\sigma_T$ is the thermal broadening corresponding to $T_0$:

$$ \sigma_T (T_0) = 9.1  \sqrt{ \frac{T_0}{10000K} } \, \mathrm{km/s} $$

Note that in the emulator $\sigma_T$ will be converted to comoving units in the same way that we have converted the linear power from velocity to comoving units.

### Simulation parameters $H$

From each simulation we run, we will output a number of snapshots, of order 10. 

Moreover, for each snapshot we extract different sets of simulated Lyman-$\alpha$ skewers (normalized quasar spectra), after applying different rescalings of the temperatures in the snapshot. 
These skewers are written to disk.

Finally, from each set of skewers we measure different 1D power spectra, after rescaling the mean optical depth in the spectra. 
This optical depth rescaling is trivial, and it is done on the fly. 
Each of these measured power spectra is fed to the emulator. 


Let's discuss the parameterization of each of the simulation packages:

- As we discuss above, the emulator labels the measured power with the set of parameters $ M = \{ M_P, f, \lambda_F, \bar F, \sigma_T, \gamma \} $.

- Each set of simulated skewers, written to disk, is described by a subset of these $ \{ M_P, f, \lambda_F, \sigma_T, \gamma \} $, since we will have different values of $\bar F$ from the skewers.

- Each snapshot is described by an even smaller subset of parameters, $ \{ M_P, f, \lambda_F \} $, since we will reprocess the snapshot for different temperature-density relations.

If we assumed that we can do as much rescaling of mean flux and temperature as we wanted, then this last set of parameters $ \{ M_P, f, \lambda_F \} $ would be the only relevant ones. If we can not rescale as much as we would like to (because the rescaling breaks down at some point?), then we would need to label the snapshots with a "central temperature" and "central mean flux", around which we would perturb. 

Now, since we are describing the linear power in comoving units, the shape of the linear power will be the same in all snapshots. The only parameters that will vary are the amplitude of the linear power, the logarithmic growth rate $f$ and the filtering length $\lambda_F$. So each simulation will be described by two shape parameters, and $N_z \times \{ A_p, f, \lambda_F \}$, where $N_z$ is the number of snapshots. 

$$ H = \{ n_p, \alpha_p, N_z \times \{ A_p, f, \lambda_F \} \} $$


The question is: how do we decide what simulation to run, i.e., what configuration files to use, if we are handed a set of parameters $H$? We will not aim at having a perfect match between the two, we just want to get on the right ball park.

From the point of view of cosmological parameters:
 - We start by defining a fiducial cosmology, somewhere around Planck, but WITHOUT NEUTRINOS. This is important, since our simulations will not have neutrinos. All quantities with $^0$ will refer to that fiducial cosmology. 
 - In flat $\Lambda$CDM models, we can translate the requirement of a given $f_\star$ into a requirement of a given $\Omega_\star = \Omega_{cb}(z_\star)$, using $f_\star = \Omega_\star^{0.55}$.
 - We can then translate the requirement on $\Omega_\star$ to a requirement on $\Omega_{cb} = \Omega_{cb}(z=0)$, using $\Omega_\star = \Omega_{cb} (1+z_\star)^3 / ( \Omega_{cb} (1+z_\star)^3 + 1 - \Omega_{cb} ) $. 
 - We will keep fixed the values of $\Omega_c h^2$ and $\Omega_b h^2$, so the requirement on $\Omega_{cb}$ will translate into a requirement on $h$, using $h^2 = (\Omega_b h^2 + \Omega_c h^2)/\Omega_{cb} $.
 - We will also measure the shape of the linear power at $z_\star$ around $k_p$, and modify the slope and running of the primordial power to match the required shape.
 - So the cosmology in the simulation will be specified by $\{ h, A_p, n_p, \alpha_p \}$, where $A_p$ will be computed at $z_\star$.

From the point of view of the pressure smoothing:
 - We will look at the value of the filtering length at a pre-HeII and at a post-HeII redshift, say z=2 and z=4. 
 - We will use the Oñorbe models to translate the required values for $\lambda_F$ pre- and post-HeII reionization to a required thermal history. It will probably be enough to specify a scaling of the heating rates before ($\mu_H$) and after ($\mu_{He}$) He_II reionizaion. 

### Initial simulation grid

We have 6 simulation parameters: 

$$ H = \{ \mu_H, \mu_{He}, h, \Delta_p^2, n_p, \alpha_p \} $$

We will choose the following parameter volume for the simulations:
 - $ 0.5 < \mu_H < 2.0 $
 - $ 0.5 < \mu_{He} < 2.0 $
 - $ 0.6 < h < 0.8 $
 - $ 0.25 < \Delta_p^2 < 0.45 $
 - $ -2.3 < n_p < -2.4 $
 - $ -0.12 < \alpha_p < -0.08 $

In [1]:
%matplotlib inline
import numpy as np
import os
import copy
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import camb
from camb import model, initialpower
import fit_pk
import read_genic

In [6]:
# setup fiducial cosmology
cosmo_fid = fit_pk.get_cosmology(mnu=0.0)
fit_pk.print_info(cosmo)
# fit power around this
z_star=3.0
kp_Mpc=0.7
# get linear power parameters, in comoving units
linP_params_fid=fit_pk.parameterize_cosmology_Mpc(cosmo_fid,z_star=z_star,kp_Mpc=kp_Mpc)
print('fiducial cosmology parameters',linP_params_fid)

H0 = 75.0 ; Omega_b h^2 = 0.022 ; Omega_c h^2 = 0.12 ; Omega_k = 0.0 ; ommnuh2 = 0.0 ; T_CMB = 2.7255 ; A_s = 2.1e-09 ; n_s = 0.96
fiducial cosmology parameters {'f_star': 0.98136698840516934, 'g_star': 0.9677508579459803, 'linP_Mpc': poly1d([-0.10748482, -2.30758676,  3.02860938])}


In [7]:
# define target cosmology
target_f_star=0.98
target_lnA_p = 2.5
target_n_p = -2.25
target_alpha_p = - 0.12